In [1]:
import numpy as np
import ray
import matplotlib.pyplot as plt
import time
from utils import *
from procedures import *
# https://albertcthomas.github.io/good-practices-random-number-generators/
# plt.rcParams["font.sans-serif"] = "Arial Unicode MS"
plt.rcParams["font.sans-serif"] = "DejaVu Sans"
# plt.rcParams["axes.unicode_minus"] = False
# mlib.rc('xtick', labelsize=15) 
# mlib.rc('ytick', labelsize=15)
plt.rcParams["mathtext.default"]  = "it"
plt.rcParams["mathtext.fontset"]  = "dejavusans"

In [2]:
num_cpus=96
ray.shutdown()
ray.init(num_cpus=num_cpus, ignore_reinit_error=True)

2023-01-18 00:30:02,756	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


Python version:,3.8.8
Ray version:,2.2.0
Dashboard:,http://127.0.0.1:8265


In [3]:
from TpMaxSimulator import TpMaxSimulator

In [4]:
recipes0 = [30, 50, 100, 200]
recipes1 = [(2, 28), (4, 46), (9, 91),(18, 182)]
recipes2 = [(27, 3), (45, 5), (90, 10),(180, 20)]
recipes3 = [(6, 21, 3), (10, 35, 5), (20, 70, 10),(40, 140, 20)]
recipes4 = [30, 50, 100, 200]

n_recipe = len(recipes0)
n_replications = 500
G = 11
_njobs, _nstages, _burnin = 1050, 3, 1000

In [5]:
FBKT_results = []
FBKTS_results = []
EFG_results = []
EFGPlus_results = []
MoSH_results = []

In [6]:
problem_types = [(20, 20)]
seed = 20230118 + 20 + 20 # date + experiment setting
np.random.seed(seed)
rng = np.random.default_rng(seed)


for problem_type in problem_types:
    slns, means = loadTPMax(problem_type[0], problem_type[1])
    print("problem size:{}".format(len(slns)))
    generators = [TpMaxGenerator(_njobs, _nstages, _burnin, slns, means, TpMaxSimulator) for i in range(n_replications)]
    for i in range(n_recipe):
        
        n = recipes0[i]
        print("FBKT")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":0, "n":n, "seeding":False})
        FBKT_results.append([PCS,PGS])
        
        recipe = recipes1[i]
        nsd = recipe[0]
        n0 = recipe[1]
        
        print("FBKTSeeding")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":nsd, "n": n0, "seeding":True})
        FBKTS_results.append([PCS,PGS])
        
        recipe = recipes2[i]
        n0 = recipe[0]
        ng = recipe[1]
        
        print("EFG")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFG, remote_policy=remote_EFG, 
                                        args={"n0":n0, "ng":ng})
        EFG_results.append([PCS,PGS])
    
        
        recipe = recipes3[i]
        nsd = recipe[0]
        n0 = recipe[1]
        ng = recipe[2]
        
        print("EFGPlus")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFGPlus, remote_policy=remote_EFGPlus,  
                                        args={"nsd":nsd, "n0":n0, "ng":ng, "G":G})

        EFGPlus_results.append([PCS,PGS])
        
        n = recipes4[i]
        if n >= 81:
            print("Modified Sequential Halving")
            PCS, PGS = parallel_experiments(rng,  generators, policy=ModifiedSH, remote_policy=remote_ModifiedSH,  args={"n":n})
            MoSH_results.append([PCS,PGS])

problem size:3249
FBKT
--------New experiments with  k=3249----------------------
Start to simulate... at Wed Jan 18 00:30:18 2023
PCS:0.314, PGS:0.778
Total time used: 97.49446177482605s, simulation ends at Wed Jan 18 00:31:55 2023
FBKTSeeding
--------New experiments with  k=3249----------------------
Start to simulate... at Wed Jan 18 00:31:56 2023
PCS:0.374, PGS:0.862
Total time used: 101.52243423461914s, simulation ends at Wed Jan 18 00:33:36 2023
EFG
--------New experiments with  k=3249----------------------
Start to simulate... at Wed Jan 18 00:33:37 2023
PCS:0.266, PGS:0.772
Total time used: 106.94088983535767s, simulation ends at Wed Jan 18 00:35:23 2023
EFGPlus
--------New experiments with  k=3249----------------------
Start to simulate... at Wed Jan 18 00:35:24 2023
PCS:0.414, PGS:0.94
Total time used: 108.89855790138245s, simulation ends at Wed Jan 18 00:37:12 2023
FBKT
--------New experiments with  k=3249----------------------
Start to simulate... at Wed Jan 18 00:37:13 202

In [7]:
problem_types = [(30, 30)]
seed = 20230118+30+30
np.random.seed(seed)
rng = np.random.default_rng(seed)


for problem_type in problem_types:
    slns, means = loadTPMax(problem_type[0], problem_type[1])
    print("problem size:{}".format(len(slns)))
    generators = [TpMaxGenerator(_njobs, _nstages, _burnin, slns, means, TpMaxSimulator) for i in range(n_replications)]
    for i in range(n_recipe):
        
        n = recipes0[i]
        print("FBKT")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":0, "n":n, "seeding":False})
        FBKT_results.append([PCS,PGS])
        
        recipe = recipes1[i]
        nsd = recipe[0]
        n0 = recipe[1]
        
        print("FBKTSeeding")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":nsd, "n": n0, "seeding":True})
        FBKTS_results.append([PCS,PGS])
        
        recipe = recipes2[i]
        n0 = recipe[0]
        ng = recipe[1]
        
        print("EFG")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFG, remote_policy=remote_EFG, 
                                        args={"n0":n0, "ng":ng})
        EFG_results.append([PCS,PGS])
    
        
        recipe = recipes3[i]
        nsd = recipe[0]
        n0 = recipe[1]
        ng = recipe[2]
        
        print("EFGPlus")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFGPlus, remote_policy=remote_EFGPlus,  
                                        args={"nsd":nsd, "n0":n0, "ng":ng, "G":G})

        EFGPlus_results.append([PCS,PGS])
        
        n = recipes4[i]
        if n >= 81:
            print("Modified Sequential Halving")
            PCS, PGS = parallel_experiments(rng,  generators, policy=ModifiedSH, remote_policy=remote_ModifiedSH,  args={"n":n})
            MoSH_results.append([PCS,PGS])

problem size:11774
FBKT
--------New experiments with  k=11774----------------------
Start to simulate... at Wed Jan 18 02:05:17 2023
PCS:0.248, PGS:0.668
Total time used: 359.16936779022217s, simulation ends at Wed Jan 18 02:11:15 2023
FBKTSeeding
--------New experiments with  k=11774----------------------
Start to simulate... at Wed Jan 18 02:11:16 2023
PCS:0.258, PGS:0.752
Total time used: 369.4780294895172s, simulation ends at Wed Jan 18 02:17:25 2023
EFG
--------New experiments with  k=11774----------------------
Start to simulate... at Wed Jan 18 02:17:26 2023
PCS:0.2, PGS:0.534
Total time used: 380.888001203537s, simulation ends at Wed Jan 18 02:23:46 2023
EFGPlus
--------New experiments with  k=11774----------------------
Start to simulate... at Wed Jan 18 02:23:47 2023
PCS:0.332, PGS:0.816
Total time used: 385.1432385444641s, simulation ends at Wed Jan 18 02:30:11 2023
FBKT
--------New experiments with  k=11774----------------------
Start to simulate... at Wed Jan 18 02:30:12 2

In [ ]:
problem_types = [(45, 30)]
seed = 20230118+45+30
np.random.seed(seed)
rng = np.random.default_rng(seed)


for problem_type in problem_types:
    slns, means = loadTPMax(problem_type[0], problem_type[1])
    print("problem size:{}".format(len(slns)))
    generators = [TpMaxGenerator(_njobs, _nstages, _burnin, slns, means, TpMaxSimulator) for i in range(n_replications)]
    for i in range(n_recipe):
        
        n = recipes0[i]
        print("FBKT")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":0, "n":n, "seeding":False})
        FBKT_results.append([PCS,PGS])
        
        recipe = recipes1[i]
        nsd = recipe[0]
        n0 = recipe[1]
        
        print("FBKTSeeding")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":nsd, "n": n0, "seeding":True})
        FBKTS_results.append([PCS,PGS])
        
        recipe = recipes2[i]
        n0 = recipe[0]
        ng = recipe[1]
        
        print("EFG")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFG, remote_policy=remote_EFG, 
                                        args={"n0":n0, "ng":ng})
        EFG_results.append([PCS,PGS])
    
        
        recipe = recipes3[i]
        nsd = recipe[0]
        n0 = recipe[1]
        ng = recipe[2]
        
        print("EFGPlus")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFGPlus, remote_policy=remote_EFGPlus,  
                                        args={"nsd":nsd, "n0":n0, "ng":ng, "G":G})

        EFGPlus_results.append([PCS,PGS])
        
        n = recipes4[i]
        if n >= 81:
            print("Modified Sequential Halving")
            PCS, PGS = parallel_experiments(rng,  generators, policy=ModifiedSH, remote_policy=remote_ModifiedSH,  args={"n":n})
            MoSH_results.append([PCS,PGS])

problem size:27434
FBKT
--------New experiments with  k=27434----------------------
Start to simulate... at Tue Jan 17 00:50:49 2023


In [ ]:
problem_types = [(45,45)]
seed = 20230118+45+45
np.random.seed(seed)
rng = np.random.default_rng(seed)


for problem_type in problem_types:
    slns, means = loadTPMax(problem_type[0], problem_type[1])
    print("problem size:{}".format(len(slns)))
    generators = [TpMaxGenerator(_njobs, _nstages, _burnin, slns, means, TpMaxSimulator) for i in range(n_replications)]
    for i in range(n_recipe):
        
        n = recipes0[i]
        print("FBKT")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":0, "n":n, "seeding":False})
        FBKT_results.append([PCS,PGS])
        
        recipe = recipes1[i]
        nsd = recipe[0]
        n0 = recipe[1]
        
        print("FBKTSeeding")
        PCS, PGS = parallel_experiments(rng,  generators, policy=FBKT, remote_policy=remote_FBKT, 
                                        args={"nsd":nsd, "n": n0, "seeding":True})
        FBKTS_results.append([PCS,PGS])
        
        recipe = recipes2[i]
        n0 = recipe[0]
        ng = recipe[1]
        
        print("EFG")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFG, remote_policy=remote_EFG, 
                                        args={"n0":n0, "ng":ng})
        EFG_results.append([PCS,PGS])
    
        
        recipe = recipes3[i]
        nsd = recipe[0]
        n0 = recipe[1]
        ng = recipe[2]
        
        print("EFGPlus")
        PCS, PGS = parallel_experiments(rng,  generators, policy=EFGPlus, remote_policy=remote_EFGPlus,  
                                        args={"nsd":nsd, "n0":n0, "ng":ng, "G":G})

        EFGPlus_results.append([PCS,PGS])
        
        n = recipes4[i]
        if n >= 81:
            print("Modified Sequential Halving")
            PCS, PGS = parallel_experiments(rng,  generators, policy=ModifiedSH, remote_policy=remote_ModifiedSH,  args={"n":n})
            MoSH_results.append([PCS,PGS])